In [1]:
from hate.settings import corpora
train = corpora['train_es']
X_train, y_train = list(train.X()), list(train.y())
dev = corpora['dev_es']
X_dev, y_dev = list(dev.X()), list(dev.y())

In [2]:
from hate.classifier import HateClassifier
filename = '/users/jsanchez/francolq/hatEval/tass2018/SENTIMENT_UBAv2_50_2.bin'
clf = HateClassifier(
    lang='es',
    bow=True,
    bow_params={
        'binary': True,
        'ngram_range': (1, 2),
    },
    boc=True,
    boc_params={
        'analyzer': 'char',
        'binary': True,
        'ngram_range': (1, 5),
    },
    #emb='fasttext',
    emb='wfasttext',
    emb_params={
        'file': filename,
    },
    clf='maxent'
)
clf.fit(X_train, y_train)
clf.eval(X_dev, y_dev)

/users/jsanchez/.virtualenvs/pln/lib/python3.7/site-packages/treetaggerwrapper.py:736: FutureWarning: Possible nested set at position 8
  re.IGNORECASE | re.VERBOSE)
/users/jsanchez/.virtualenvs/pln/lib/python3.7/site-packages/treetaggerwrapper.py:2025: FutureWarning: Possible nested set at position 152
  re.VERBOSE | re.IGNORECASE)
/users/jsanchez/.virtualenvs/pln/lib/python3.7/site-packages/treetaggerwrapper.py:2048: FutureWarning: Possible nested set at position 409
  UrlMatch_re = re.compile(UrlMatch_expression, re.VERBOSE | re.IGNORECASE)
/users/jsanchez/.virtualenvs/pln/lib/python3.7/site-packages/treetaggerwrapper.py:2060: FutureWarning: Possible nested set at position 192
  EmailMatch_re = re.compile(EmailMatch_expression, re.VERBOSE | re.IGNORECASE)
/users/jsanchez/.virtualenvs/pln/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


accuracy	0.85

              precision    recall  f1-score   support

           0       0.87      0.85      0.86       278
           1       0.82      0.84      0.83       222

   micro avg       0.85      0.85      0.85       500
   macro avg       0.85      0.85      0.85       500
weighted avg       0.85      0.85      0.85       500

[[237  41]
 [ 35 187]]


In [5]:
import pandas as pd
pd.options.display.max_colwidth = 0

y_pred = clf.predict(X_dev)
y_proba = clf.predict_proba(X_dev)
errors = []
false_positives, false_negatives = [], []
for i, (x, y1, y2, y2p) in enumerate(zip(X_dev, y_dev, y_pred, y_proba)):
    if y1 != y2:
        diff = y2p[int(y1)] - y2p[int(y2)]
        errors.append({
            'index': i,
            'item': x,
            'true': y1,
            'pred': y2,
            'pneg': y2p[0],
            'ppos': y2p[1],
            'diff': diff})
        if (y1, y2) == ('0', '1'):
            false_positives.append(i)
        else:
            false_negatives.append(i)

errdf = pd.DataFrame(errors)
errdf.sort_values('diff', inplace=True)

In [6]:
errdf[:10]

,diff,index,item,pneg,ppos,pred,true
31,-0.930172,214,Ni tú eres una zorra hambrienta ni yo un incorregible depredador.,0.034914,0.965086,1,0
33,-0.909822,222,"Una chica no es una guarra, una zorra, una puta, una buscan por su manera de vestir.",0.045089,0.954911,1,0
36,-0.871188,256,"De Colombia tocará emigrar en pateras, como hacen los africanos subsaharianos.",0.064406,0.935594,1,0
7,-0.852876,37,"Zorra tu abuela, imbécil.",0.073562,0.926438,1,0
29,-0.836379,197,@saancisco Callate puta tu eres el traidor.,0.081810,0.918190,1,0
62,-0.833748,422,"@LeonDeJapon Jajajajajaja tu eres uno, diego jajajajajaja mentira perra perra perrraaaaaaaaaa de mierda jajaja ♡",0.083126,0.916874,1,0
32,-0.758311,219,| lo único que dominas es mi polla en tu cara. https://t.co/qlYOd4zPwW,0.879155,0.120845,0,1
20,-0.729265,161,Talvez quede como zorra pero te lo mereces Socia😎,0.135367,0.864633,1,0
11,-0.680665,104,@NicolasMaduro Callate la jeta perra que de este año no pasas,0.159667,0.840333,1,0
1,-0.670143,15,Racismo es que hagas enfrijoladas con frijoles güeros y no negros.,0.164928,0.835072,1,0


In [7]:
false_positives, false_negatives

([2,
  15,
  20,
  37,
  84,
  98,
  99,
  104,
  112,
  116,
  134,
  161,
  169,
  176,
  179,
  182,
  186,
  197,
  209,
  214,
  222,
  248,
  256,
  259,
  270,
  293,
  313,
  328,
  332,
  342,
  360,
  366,
  389,
  406,
  416,
  422,
  425,
  428,
  447,
  457,
  468],
 [21,
  25,
  33,
  35,
  114,
  129,
  148,
  154,
  158,
  162,
  181,
  196,
  219,
  231,
  262,
  269,
  301,
  305,
  318,
  326,
  348,
  380,
  393,
  401,
  413,
  415,
  418,
  439,
  450,
  455,
  460,
  482,
  488,
  493,
  494])

## Primer Error: Negación 'ni'

In [35]:
# errors[0]['item']
#tweet = errdf.values[0,2]
tweet = 'Ni tú eres una zorra hambrienta ni yo un incorregible depredador.'
tweet = 'No tú eres una zorra hambrienta ni yo un incorregible depredador.'
#from hate.tokenizer import Tokenizer
tkn = clf.build_bow_tokenizer()
tkn(tweet)

['no', 'NOT_zorra', 'NOT_hambrienta', 'NOT_incorregible', 'depredador']

Agregamos 'ni' a la lista de tokens de negación.

Esa instancia se arregla pero otras cinco se rompen.

In [2]:
from hate.classifier import HateClassifier
filename = '/users/jsanchez/francolq/hatEval/tass2018/SENTIMENT_UBAv2_50_2.bin'
clf = HateClassifier(
    lang='es',
    bow=True,
    bow_params={
        'binary': True,
        'ngram_range': (1, 2),
    },
    boc=True,
    boc_params={
        'analyzer': 'char',
        'binary': True,
        'ngram_range': (1, 5),
    },
    #emb='fasttext',
    emb='wfasttext',
    emb_params={
        'file': filename,
    },
    clf='maxent'
)
clf.fit(X_train, y_train)
clf.eval(X_dev, y_dev)

/users/jsanchez/.virtualenvs/pln/lib/python3.7/site-packages/treetaggerwrapper.py:736: FutureWarning: Possible nested set at position 8
  re.IGNORECASE | re.VERBOSE)
/users/jsanchez/.virtualenvs/pln/lib/python3.7/site-packages/treetaggerwrapper.py:2025: FutureWarning: Possible nested set at position 152
  re.VERBOSE | re.IGNORECASE)
/users/jsanchez/.virtualenvs/pln/lib/python3.7/site-packages/treetaggerwrapper.py:2048: FutureWarning: Possible nested set at position 409
  UrlMatch_re = re.compile(UrlMatch_expression, re.VERBOSE | re.IGNORECASE)
/users/jsanchez/.virtualenvs/pln/lib/python3.7/site-packages/treetaggerwrapper.py:2060: FutureWarning: Possible nested set at position 192
  EmailMatch_re = re.compile(EmailMatch_expression, re.VERBOSE | re.IGNORECASE)
/users/jsanchez/.virtualenvs/pln/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


accuracy	0.84

              precision    recall  f1-score   support

           0       0.86      0.85      0.85       278
           1       0.82      0.82      0.82       222

   micro avg       0.84      0.84      0.84       500
   macro avg       0.84      0.84      0.84       500
weighted avg       0.84      0.84      0.84       500

[[237  41]
 [ 40 182]]


In [3]:
import pandas as pd
pd.options.display.max_colwidth = 0

y_pred = clf.predict(X_dev)
y_proba = clf.predict_proba(X_dev)
errors = []
false_positives, false_negatives = [], []
for i, (x, y1, y2, y2p) in enumerate(zip(X_dev, y_dev, y_pred, y_proba)):
    if y1 != y2:
        diff = y2p[int(y1)] - y2p[int(y2)]
        errors.append({
            'index': i,
            'item': x,
            'true': y1,
            'pred': y2,
            'pneg': y2p[0],
            'ppos': y2p[1],
            'diff': diff})
        if (y1, y2) == ('0', '1'):
            false_positives.append(i)
        else:
            false_negatives.append(i)

errdf = pd.DataFrame(errors)
errdf.sort_values('diff', inplace=True)

In [7]:
old_false_positives, old_false_negatives = ([2,
  15,
  20,
  37,
  84,
  98,
  99,
  104,
  112,
  116,
  134,
  161,
  169,
  176,
  179,
  182,
  186,
  197,
  209,
  214,
  222,
  248,
  256,
  259,
  270,
  293,
  313,
  328,
  332,
  342,
  360,
  366,
  389,
  406,
  416,
  422,
  425,
  428,
  447,
  457,
  468],
 [21,
  25,
  33,
  35,
  114,
  129,
  148,
  154,
  158,
  162,
  181,
  196,
  219,
  231,
  262,
  269,
  301,
  305,
  318,
  326,
  348,
  380,
  393,
  401,
  413,
  415,
  418,
  439,
  450,
  455,
  460,
  482,
  488,
  493,
  494])

In [13]:
# len(false_positives), len(false_negatives)
new_errors = set(false_negatives) - set(old_false_negatives)
#[X_dev[i] for i in new_errors]

In [14]:
tkn = clf.build_bow_tokenizer()
pd.DataFrame([(X_dev[i], tkn(X_dev[i])) for i in new_errors])

,0,1
0,@CarlaEUrena @FelipeCalderon Callateeeee !! Puta gallina cacarea !! Ni idea tienes de nada .,"[@user, callateee, puta, gallina, cacarea, ni, NOT_idea, NOT_tener]"
1,"@Stop2Invasion la nacionalidad mal otorgada y con comportamientos no adientes y sin voluntad de integrarse etc..se puede retrotraer, así que menos happy inmigrantes porque les queda lo que se quede Sanchez","[@user, nacionalidad, mal, otorgado, comportamiento, no, NOT_adientes, NOT_voluntad, NOT_integrar, etcétera, poder, retrotraer, así, menos, happy, inmigrante, quedar, quedar, sanchez]"
2,"La fundación Francisco Franco propone eliminar las concertinas de la valla de Ceuta, y untar con tocino de cerdo la frontera para evitar que los moros nos coman.","[fundación, francisco, franco, proponer, eliminar, concertinas, valla, ceuta, untar, tocino, cerdo, frontera, evitar, moro, coman]"
3,"Ninguna mujer es tan santa como para no aprender a coger bien, ni tan puta como para no aprender a amar.","[ninguno, mujer, santo, no, NOT_aprender, NOT_coger, NOT_bien, ni, NOT_puta, no, NOT_aprender, NOT_amar]"
4,"Q pasa en🇪🇸,Q tenemos sindicatos de manteros ,como se puede estar sindicado sin tener papeles ni pasaportes ni nada Q acrediten de donde son y quién son https://t.co/gzsAZUd8xA","[q, pasar, 🇪, 🇸, q, tener, sindicato, manteros, poder, sindicado, tener, papel, ni, NOT_pasaporte, ni, NOT_q, NOT_acrediten, NOT_quién, url]"


In [5]:
errdf[:10]

,diff,index,item,pneg,ppos,pred,true
35,-0.913137,222,"Una chica no es una guarra, una zorra, una puta, una buscan por su manera de vestir.",0.043432,0.956568,1,0
38,-0.870755,256,"De Colombia tocará emigrar en pateras, como hacen los africanos subsaharianos.",0.064623,0.935377,1,0
7,-0.854502,37,"Zorra tu abuela, imbécil.",0.072749,0.927251,1,0
31,-0.836161,197,@saancisco Callate puta tu eres el traidor.,0.081919,0.918081,1,0
66,-0.833299,422,"@LeonDeJapon Jajajajajaja tu eres uno, diego jajajajajaja mentira perra perra perrraaaaaaaaaa de mierda jajaja ♡",0.083351,0.916649,1,0
34,-0.747013,219,| lo único que dominas es mi polla en tu cara. https://t.co/qlYOd4zPwW,0.873506,0.126494,0,1
22,-0.723486,161,Talvez quede como zorra pero te lo mereces Socia😎,0.138257,0.861743,1,0
70,-0.707617,439,"¿Ves que tu coño me importe? Cómete mi polla, basura https://t.co/ht02KyYWZh",0.853808,0.146192,0,1
1,-0.688847,15,Racismo es que hagas enfrijoladas con frijoles güeros y no negros.,0.155577,0.844423,1,0
11,-0.687693,104,@NicolasMaduro Callate la jeta perra que de este año no pasas,0.156154,0.843846,1,0
